<a href="https://colab.research.google.com/github/carloslme/ds-portfolio/blob/main/predictive-analytics/predictive_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Predictive Analytics is a process of statistical techniques derived from data mining, machine learning and predictive modeling that obtain current and historical events to predict future events or unknown outcomes in the future.

The goal of this exercise is to build a machine learning pipeline for predictive analytics using this open dataset: https://datos.cdmx.gob.mx/dataset/incidentes-viales-c5, which shows the road incidents in Mexico City during february 2021 in the 16 town halls.

# Data Ingestion

## Getting data from website 

In [ ]:
!pip install requests

In [2]:
import requests

In [3]:
res = requests.get('https://datos.cdmx.gob.mx/dataset/a6d1d483-65d2-4ed9-9687-1932eb0cf70f/resource/c6b0fe65-137c-40d6-99c9-6c527f9b29c4/download/___')

In [8]:
res.raise_for_status()
csv_file = open('incidentes-viales-c5.csv', 'wb')
for chunk in res.iter_content(100000):
  csv_file.write(chunk)

csv_file.close()

# Data Exploration 

In [9]:
import pandas as pd

In [10]:
incidents = pd.read_csv('/content/incidentes-viales-c5.csv')
incidents.head()

,Unnamed: 0,folio,fecha_creacion,hora_creacion,dia_semana,fecha_cierre,hora_cierre,incidente_c4,delegacion_inicio,latitud,longitud,codigo_cierre,clas_con_f_alarma,tipo_entrada,delegacion_cierre,ano,mes
0,1,C5/210201/04663,01/02/2021,19:14:39,Lunes,01/02/2021,20:36:48,accidente-choque con lesionados,GUSTAVO A. MADERO,19.461570,-99.115210,A,URGENCIAS MEDICAS,LLAMADA DEL 911,GUSTAVO A. MADERO,2021,2
1,2,C5/210201/00391,01/02/2021,01:02:02,Lunes,01/02/2021,01:20:09,accidente-choque sin lesionados,GUSTAVO A. MADERO,19.471440,-99.081770,N,EMERGENCIA,LLAMADA DEL 911,GUSTAVO A. MADERO,2021,2
2,3,C5/210201/03815,01/02/2021,16:49:58,Lunes,01/02/2021,19:11:53,accidente-choque con lesionados,MIGUEL HIDALGO,19.440680,-99.190980,A,URGENCIAS MEDICAS,LLAMADA DEL 911,MIGUEL HIDALGO,2021,2
3,4,C5/210201/02068,01/02/2021,11:10:52,Lunes,01/02/2021,12:24:56,accidente-choque sin lesionados,COYOACAN,19.342580,-99.148890,A,EMERGENCIA,LLAMADA DEL 911,COYOACAN,2021,2
4,5,C5/210201/03832,01/02/2021,16:52:41,Lunes,01/02/2021,17:38:14,accidente-choque con lesionados,TLALPAN,19.200021,-99.138111,N,URGENCIAS MEDICAS,LLAMADA DEL 911,TLALPAN,2021,2


# Data Preprocessing

# Modeling

# Report

# Deployment and consumption

# Useful resources
* https://www.educba.com/predictive-analysis-vs-forecasting/?source=leftnav